In [9]:
import requests, bs4, selenium, math, time, functools
from PyPDF2 import PdfFileReader
from selenium import webdriver

In [10]:
PATH = './driver/chromedriver.exe'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36'
}
pref = 'https://ocw.mit.edu'

In [11]:
class CourseRes:
    
    def __init__(self, nome, link, durata, course):
        self.idN = course[0]
        self.link=link
        self.discip=course[1]
        self.kywTit=course[2]
        self.nome=nome
        self.desc = course[3]
        self.typeR="presentation"
        self.language="English"
        self.difficulty = course[4]
        def setCommonVal():
            self.formato="pdf"
            roundedRis=math.floor(durata/30)
            if roundedRis<120:
                self.durata = "[{}-{}]".format(*[roundedRis*30, (roundedRis+1)*30])
            else:
                self.durata = "[120+]"
        if "undergraduate" in self.difficulty.lower():
            setCommonVal()
            self.public = ['Middle School', 'High School']
            self.min_age=11
            self.max_age=18
        else:
            setCommonVal()
            self.public = ['College General Ed', 'Graduate School', 'Professional']
            self.min_age=19
            self.max_age=100
    
    def printInCSV(self, file):
        for attr in self.__dict__:
            item = self.__dict__[attr]
            file.write("{};".format(item))
        file.write("\n")
        

In [12]:
def datacollector (link, f, driver):
    driver.get(link)
    html = driver.page_source
    soup=bs4.BeautifulSoup(html, "html.parser")
    des = soup.find_all("div", {"id" : "course-description"})[0].text.replace("\n", " ").replace(";", ",")
    lvl = soup.find_all("a", {"class" : "course-info-level"})[0].text.replace(" ", "").replace("\n", "")
    tit = soup.find_all("a", {"class" : "text-uppercase display-4 font-weight-bold m-0 text-white"})[0].text.replace("\n", "")
    ids = [li.text.replace(" ", "").replace("\n", "") for li in soup.find_all("ul", {"class" : "list-unstyled m-0"})[1].find_all("li")]
    idl = functools.reduce(lambda x, y: x+","+y , ids, "")[1:]
    t = soup.find_all("table", {"id" : "course-info-table"})
    topic = ""
    for c in t:
        for a in c.find_all("ul", {"class" : "list-unstyled pb-2 m-0"}):
            for g in a.find_all("li"):
                topic += g.text.replace("\n", "").replace ("      >          ", "/")+", "
    course = [idl, topic, tit, des, lvl]
    patternlink= link[len(pref):]+"pages/lecture-notes"
    if bs4.BeautifulSoup(html, "html.parser").find_all("a", {"href" : patternlink}):
        nlink = pref + patternlink
        getCoursePDF(nlink, f, course, driver) 
        del course
        print("Downloaded files from "+nlink)
    

In [13]:
def getCoursePDF(link, f , course, driver):
    driver.get(link)
    html = driver.page_source
    soup = bs4.BeautifulSoup(html, "html.parser").find_all("div", {"id" : "main-content"})
    for s1 in soup:
        for s in s1.find_all("td"):
            for e in s.find_all("a"):
                nlink=pref+e.attrs["href"]
            
                response = requests.get(nlink,{"headers" : headers})
                href = bs4.BeautifulSoup(response.text, "html.parser").find("a", {"class":"download-file"})
                if href["href"][-3:]=="pdf":
                    pdflink = pref+href["href"]
                    response = requests.get(pdflink, stream=True)
                    my_raw_data = response.content
                
                    with open("my_pdf.pdf", 'wb') as my_data:
                        my_data.write(my_raw_data) 
                    
                        open_pdf_file = open("my_pdf.pdf", 'rb')
                        read_pdf = PdfFileReader(open_pdf_file)
                        obj=CourseRes(str(read_pdf.getDocumentInfo().title), pdflink, read_pdf.getNumPages(), course)
                        open_pdf_file.close()
                        #Metti qui il codice per caricare il file sul drive. Ricordati di cambiare nome al pdf
                        obj.printInCSV(f)
                        del obj

In [14]:
urlfine = "https://ocw.mit.edu/courses/wgs-s10-history-of-women-in-science-and-engineering-fall-2017/"
url = 'https://ocw.mit.edu/search/?s=department_course_numbers.sort_coursenum'
pref="https://ocw.mit.edu"
#fineurl = 'https://ocw.mit.edu/courses/1-34-waste-containment-and-remediation-technology-spring-2004/'
PATH = './driver/chromedriver.exe'
driver = webdriver.Chrome(PATH)
driver.get(url)
time.sleep(5)
html = driver.page_source
ntot = int(bs4.BeautifulSoup(html, 'html.parser').find_all("span", {"class" : "results-total-number"})[0].text)
oldlinks=[]
f = open('res_dataset.csv', 'wt')
f.write("id;link;disciplines;keywords;title;description;type;language;difficulty;format;duration;public;min_age;max_age\n")
f.close()
f=open('res_dataset.csv', 'a')
while ntot>len(oldlinks):
    html = driver.page_source
    links = set([l for l in bs4.BeautifulSoup(html, 'html.parser').find_all("a", {"class" : "w-100"}) if not (l in oldlinks)])
    for s in links:
        datacollector("https://ocw.mit.edu" + s.attrs["href"], f, driver)
    driver.execute_script("window.scrollBy(0,500)")
    oldlinks=oldlinks + [l for l in links if not (l in oldlinks)]
driver.quit()
f.close()

Downloaded files from https://ocw.mit.edu/courses/1-040-project-management-spring-2004/pages/lecture-notes
Downloaded files from https://ocw.mit.edu/courses/1-00-introduction-to-computers-and-engineering-problem-solving-spring-2012/pages/lecture-notes
Downloaded files from https://ocw.mit.edu/courses/1-022-introduction-to-network-models-fall-2018/pages/lecture-notes
Downloaded files from https://ocw.mit.edu/courses/1-011-project-evaluation-spring-2011/pages/lecture-notes
Downloaded files from https://ocw.mit.edu/courses/1-018j-ecology-i-the-earth-system-fall-2009/pages/lecture-notes
Downloaded files from https://ocw.mit.edu/courses/1-017-computing-and-data-analysis-for-environmental-applications-fall-2003/pages/lecture-notes
Downloaded files from https://ocw.mit.edu/courses/1-020-ecology-ii-engineering-for-sustainability-spring-2008/pages/lecture-notes
Downloaded files from https://ocw.mit.edu/courses/1-033-mechanics-of-material-systems-an-energy-approach-fall-2003/pages/lecture-notes


WebDriverException: Message: chrome not reachable
  (Session info: chrome=100.0.4896.75)
Stacktrace:
Backtrace:
	Ordinal0 [0x007E9943+2595139]
	Ordinal0 [0x0077C9F1+2148849]
	Ordinal0 [0x006743F0+1065968]
	Ordinal0 [0x006687C2+1017794]
	Ordinal0 [0x00668FF8+1019896]
	Ordinal0 [0x0066A892+1026194]
	Ordinal0 [0x00664219+999961]
	Ordinal0 [0x00675860+1071200]
	Ordinal0 [0x006CB2D2+1422034]
	Ordinal0 [0x006BB806+1357830]
	Ordinal0 [0x00696086+1204358]
	Ordinal0 [0x00696F96+1208214]
	GetHandleVerifier [0x0098B232+1658114]
	GetHandleVerifier [0x00A4312C+2411516]
	GetHandleVerifier [0x0087F261+560433]
	GetHandleVerifier [0x0087E366+556598]
	Ordinal0 [0x0078286B+2173035]
	Ordinal0 [0x007875F8+2192888]
	Ordinal0 [0x007876E5+2193125]
	Ordinal0 [0x007911FC+2232828]
	BaseThreadInitThunk [0x7648FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77CA7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77CA7A4E+238]
